In [213]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [357]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train_Id=train['Id']
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [358]:
print('training data dimensions: ',np.shape(train))
print('test data dimensions: ',np.shape(test))

training data dimensions:  (1460, 81)
test data dimensions:  (1459, 80)


In [359]:
train.isna().sum().sort_values(ascending = False)

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
                 ... 
CentralAir          0
SaleCondition       0
Heating             0
TotalBsmtSF         0
Id                  0
Length: 81, dtype: int64

In [360]:
#Drop MACR variables
train = train.drop(['PoolQC','MiscFeature','Alley','Fence'], axis = 1)

In [361]:
#Prevent the original data from being lost
train_filling = train.copy()

In [362]:
train_filling['Electrical'].value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [415]:
train_filling['BsmtExposure'].value_counts()

No             953
Av             221
Gd             134
Mn             114
no response     38
Name: BsmtExposure, dtype: int64

In [363]:
#deal with missing value by filling no response

filling_no_response=['FireplaceQu','GarageType','GarageCond','BsmtFinType1','BsmtQual',
               'GarageFinish','GarageQual','BsmtFinType2','BsmtCond',
               'BsmtExposure']
train_filling[filling_no_response]=train_filling[filling_no_response].fillna('no response')

In [364]:
#deal with missing value by filling the average column value
train_filling['LotFrontage'].fillna(train_filling['LotFrontage'].mean(), inplace = True)
train_filling['GarageYrBlt'].fillna(train_filling['GarageYrBlt'].mean(), inplace = True)
train_filling['MasVnrArea'].fillna(train_filling['MasVnrArea'].mean(), inplace = True)

In [365]:
train_filling.isna().sum().sort_values(ascending = False)

MasVnrType      8
Electrical      1
SalePrice       0
Foundation      0
RoofMatl        0
               ..
KitchenQual     0
TotRmsAbvGrd    0
Functional      0
Fireplaces      0
Id              0
Length: 77, dtype: int64

In [366]:
train_filling['MasVnrType'].value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [367]:
train_filling['Electrical'].value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [368]:
train_filling['MasVnrType'].mode()

0    None
dtype: object

In [369]:
train_filling['Electrical'].mode()

0    SBrkr
dtype: object

In [370]:
#deal with missing value by filling the most frequently value
train_filling['MasVnrType'].fillna('None', inplace = True)
train_filling['Electrical'].fillna('SErKr', inplace = True)

In [371]:
train_filling.isna().sum().sort_values(ascending = False)

SalePrice       0
ExterCond       0
RoofStyle       0
RoofMatl        0
Exterior1st     0
               ..
KitchenAbvGr    0
KitchenQual     0
TotRmsAbvGrd    0
Functional      0
Id              0
Length: 77, dtype: int64

In [372]:
test.isna().sum().sort_values(ascending = False)

PoolQC         1456
MiscFeature    1408
Alley          1352
Fence          1169
FireplaceQu     730
               ... 
Electrical        0
CentralAir        0
HeatingQC         0
Foundation        0
Id                0
Length: 80, dtype: int64

In [373]:
#Drop MACR variables
test = test.drop(['PoolQC','MiscFeature','Alley','Fence'], axis = 1)

In [374]:
#Prevent the original data from being lost
test_filling = test.copy()
test_filling

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,120,0,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,...,34,0,0,0,0,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,...,36,0,0,0,0,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,...,82,0,0,144,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,...,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,...,32,0,0,0,0,700,7,2006,WD,Normal


In [375]:
#deal with missing value by filling no response

filling_no_response=['FireplaceQu','GarageType','GarageCond','BsmtFinType1','BsmtQual','Utilities',
               'GarageFinish','GarageQual','BsmtFinType2','BsmtCond','Functional','Exterior1st',
               'BsmtExposure','MasVnrType','Electrical','MSZoning','Exterior2nd']
test_filling[filling_no_response]=test_filling[filling_no_response].fillna('no response')

In [376]:
#deal with missing value by filling the average column value

filling_mean=['LotFrontage','GarageYrBlt','MasVnrArea','BsmtUnfSF','TotalBsmtSF','BsmtFinSF1',
               'GarageArea','BsmtFinSF2']
test_filling[filling_mean]=test_filling[filling_mean].fillna(test_filling[filling_mean].mean())

In [377]:
test_filling.isna().sum().sort_values(ascending=False)

BsmtFullBath    2
BsmtHalfBath    2
SaleType        1
KitchenQual     1
GarageCars      1
               ..
BedroomAbvGr    0
KitchenAbvGr    0
TotRmsAbvGrd    0
Functional      0
Id              0
Length: 76, dtype: int64

In [393]:
test_filling['BsmtFullBath'].value_counts()

0.0    849
1.0    584
2.0     23
3.0      1
Name: BsmtFullBath, dtype: int64

In [394]:
test_filling['BsmtHalfBath'].value_counts()

0.0    1364
1.0      91
2.0       2
Name: BsmtHalfBath, dtype: int64

In [395]:
test_filling['SaleType'].value_counts()

WD       1258
New       117
COD        44
ConLD      17
CWD         8
Oth         4
ConLI       4
Con         3
ConLw       3
Name: SaleType, dtype: int64

In [396]:
test_filling['KitchenQual'].value_counts()

TA    757
Gd    565
Ex    105
Fa     31
Name: KitchenQual, dtype: int64

In [407]:
test_filling['GarageCars'].value_counts()

2.0    770
1.0    407
3.0    193
0.0     76
4.0     11
2        1
5.0      1
Name: GarageCars, dtype: int64

In [408]:
test_filling['BsmtFullBath'].mode()

0    0
dtype: object

In [409]:
test_filling['BsmtHalfBath'].mode()

0    0
dtype: object

In [410]:
test_filling['SaleType'].mode()

0    WD
dtype: object

In [411]:
test_filling['KitchenQual'].mode()

0    TA
dtype: object

In [412]:
test_filling['GarageCars'].mode()

0    2
dtype: object

In [413]:
#deal with missing value by filling the most frequently value

test_filling['BsmtFullBath'].fillna('0.0', inplace = True)
test_filling['BsmtHalfBath'].fillna('0.0', inplace = True)
test_filling['SaleType'].fillna('WD', inplace = True)
test_filling['KitchenQual'].fillna('TA', inplace = True)
test_filling['GarageCars'].fillna('2', inplace = True)

In [414]:
test_filling.isna().sum().sort_values(ascending = False)

SaleCondition    0
ExterCond        0
RoofStyle        0
RoofMatl         0
Exterior1st      0
                ..
KitchenAbvGr     0
KitchenQual      0
TotRmsAbvGrd     0
Functional       0
Id               0
Length: 76, dtype: int64